In [1]:
# Import libraries
import numpy as np 
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Prepare Dataset
# load data

# relative path to npz files
path = 'Measurements'
file_name = 'output_batch_%d.npz'

# Training
# how many frames to load
frames_num = 10
data_frames = []
data_forces = []
for i in range(frames_num):
    file_path = os.path.join(path,file_name %i)
    data = np.load(file_path)
    data_frames.append(data['frames'])
    data_forces.append(data['forces'])

combined_frames = np.concatenate(data_frames, axis=0)
combined_forces = np.concatenate(data_forces, axis=0)

# Test
# file_path = os.path.join(path, file_name %11)
# test_data = np.load(file_path)

In [3]:
# train test split. Size of train data is 10/11 and size of test data is 1/11 
# features_train, features_test, targets_train, targets_test = train_test_split(combined_frames,
#                                                                              combined_forces[:,2],
#                                                                              test_size = (1/11),
#                                                                              random_state = 42) 
# train_test_split NOT RECOMMENDED, because it shuffles temporally dependent data
features_train = combined_frames
targets_train = combined_forces[:,2]
# features_test = test_data['frames']
# targets_test = test_data['forces']
# targets_test = targets_test[:,2]

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.from_numpy(features_train)
targetsTrain = torch.from_numpy(targets_train)

# create feature and targets tensor for test set. 
# featuresTest = torch.from_numpy(features_test)
# targetsTest = torch.from_numpy(targets_test)

In [4]:
# Create RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_channels, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(input_channels, 9, kernel_size = 5, stride = 1, padding = 2)
        self.pool = nn.MaxPool2d(kernel_size = 4, stride = 4)
        self.conv2 = nn.Conv2d(9, 18, kernel_size = 5, stride = 1, padding = 2) # If needed
        
        # RNN
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(18 * 16 * 30, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Convolutional layers
        x = self.pool(nn.ELU()(self.conv1(x)))
        x = self.pool(nn.ELU()(self.conv2(x))) # If needed
        
        # Reshape for RNN
        x = torch.reshape(x, (-1, 1, 18 * 16 * 30))  # Reshape to (batch_size, seq_len, input_size)
        print("Size of x:", x.size())  # Print size of x
        
        # RNN
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        #print("Size of h0:", h0.size())  # Print size of h0 - Debugging - Obsolete
        out, hn = self.rnn(x, h0)
        
        # Output layer
        out = self.fc(out[:, -1, :]) 
        return out


In [5]:
# batch_size, epoch and iteration
batch_size = 1000
num_epochs = 2

# Pytorch train and test sets
train = TensorDataset(featuresTrain,targetsTrain)
# test = TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
# test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)
    
# Create RNN
input_channels = 3  # RGB channels
hidden_dim = 100  # hidden layer dimension
layer_dim = 1     # number of hidden layers
output_dim = 1   # output dimension

model = RNNModel(input_channels, hidden_dim, layer_dim, output_dim)

# Define your loss function
error = nn.MSELoss()

# Define your optimizer
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# Training
seq_dim = 50 # Consider the whole batch to be temporally correlated
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    print('~~~BEGINNING OF DATASET~~~')
    for i, (images, labels) in enumerate(train_loader):
        images = images.float()
        # print(images.shape)  # Add this line to check the shape of images - Debugging purposes
        images = images.permute(0, 3, 1, 2)
            
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(images)
        outputs = torch.squeeze(outputs)
        
        # Calculate loss
        loss = error(outputs, labels.float())
        
        # Backpropagation
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
            
        # Store loss and iteration
        loss_list.append(loss.data)
        # Print Loss
        if count % 1 == 0: # for now print for every iteration
            print('Iteration: {}  Loss: {}'.format(count, loss.data.item()))

~~~BEGINNING OF DATASET~~~
Size of x: torch.Size([1000, 1, 8640])
Iteration: 1  Loss: 23510.298828125
Size of x: torch.Size([1000, 1, 8640])
Iteration: 2  Loss: 33474.90234375
Size of x: torch.Size([1000, 1, 8640])
Iteration: 3  Loss: 1232.0384521484375
Size of x: torch.Size([1000, 1, 8640])
Iteration: 4  Loss: 10396.943359375
Size of x: torch.Size([1000, 1, 8640])
Iteration: 5  Loss: 1861.7091064453125
Size of x: torch.Size([1000, 1, 8640])
Iteration: 6  Loss: 1363.58251953125
Size of x: torch.Size([1000, 1, 8640])
Iteration: 7  Loss: 2874.8125
Size of x: torch.Size([1000, 1, 8640])
Iteration: 8  Loss: 1053.53759765625
Size of x: torch.Size([1000, 1, 8640])
Iteration: 9  Loss: 2.760005235671997
Size of x: torch.Size([1000, 1, 8640])
Iteration: 10  Loss: 855.9058227539062
~~~BEGINNING OF DATASET~~~
Size of x: torch.Size([1000, 1, 8640])
Iteration: 11  Loss: 1306.25439453125
Size of x: torch.Size([1000, 1, 8640])
Iteration: 12  Loss: 660.7703857421875
Size of x: torch.Size([1000, 1, 864

In [7]:
print(loss_list)
print(np.mean(loss_list))

[tensor(23510.2988), tensor(33474.9023), tensor(1232.0385), tensor(10396.9434), tensor(1861.7091), tensor(1363.5825), tensor(2874.8125), tensor(1053.5376), tensor(2.7600), tensor(855.9058), tensor(1306.2544), tensor(660.7704), tensor(101.9136), tensor(54.3329), tensor(286.3188), tensor(24.4982), tensor(276.5011), tensor(17.3119), tensor(104.3778), tensor(135.1384)]
3979.6953


In [8]:
print(outputs) # Small sanity check

tensor([-12.0389, -12.2216, -12.4135, -12.9569, -14.0069, -13.2281, -13.5223,
        -12.4763, -12.2662, -12.5269, -12.4992, -13.2178, -13.4738, -14.2197,
        -13.4768, -13.3302, -13.6710, -13.1108, -13.9073, -13.9472, -14.5618,
        -13.1335, -13.0651, -12.7524, -12.3736, -12.1626, -13.2127, -12.8865,
        -12.6148, -12.9973, -13.1842, -13.6202, -13.7111, -12.5904, -12.9495,
        -12.4834, -12.6316, -12.0620, -12.9607, -13.8921, -13.6567, -13.6668,
        -13.8833, -14.1457, -13.5324, -13.4590, -13.9893, -13.6739, -13.4093,
        -13.5735, -12.3026, -13.0420, -13.6567, -12.9436, -13.1029, -12.6699,
        -12.9527, -12.8513, -13.2913, -12.6116, -12.6981, -13.2309, -13.1655,
        -13.4798, -13.0522, -13.6539, -13.9016, -13.5796, -13.2757, -13.5511,
        -14.3168, -13.7013, -14.0783, -12.7724, -13.0944, -12.1325, -12.4959,
        -13.0983, -12.8399, -13.4726, -13.1921, -13.1825, -13.6478, -12.9765,
        -13.0410, -12.4911, -12.8078, -13.9186, -13.9956, -13.20

In [9]:
print(labels.float())

tensor([-1.0797, -0.9949, -0.9238, -0.8581, -0.7988, -0.7402, -0.6864, -0.6413,
        -0.6007, -0.5637, -0.5303, -0.5034, -0.4781, -0.4516, -0.4438, -0.4478,
        -0.4525, -0.4622, -0.4775, -0.4831, -0.4824, -0.4824, -0.4840, -0.4829,
        -0.4796, -0.4784, -0.4745, -0.4723, -0.4696, -0.4686, -0.4738, -0.4907,
        -0.5179, -0.5654, -0.6264, -0.6943, -0.7702, -0.8488, -0.9312, -1.0187,
        -1.1152, -1.2284, -1.3486, -1.4846, -1.6458, -1.8335, -2.0371, -2.2583,
        -2.4803, -2.6908, -2.8747, -3.0401, -3.1835, -3.2991, -3.3773, -3.4140,
        -3.4004, -3.3335, -3.2181, -3.0745, -2.9110, -2.7374, -2.5585, -2.3805,
        -2.1968, -2.0178, -1.8431, -1.6839, -1.5378, -1.3989, -1.2665, -1.1307,
        -0.9906, -0.8688, -0.7793, -0.7269, -0.7026, -0.7065, -0.7217, -0.7390,
        -0.7422, -0.7402, -0.7296, -0.7031, -0.6639, -0.6237, -0.5844, -0.5494,
        -0.5165, -0.4928, -0.4744, -0.4658, -0.4598, -0.4682, -0.4804, -0.4992,
        -0.5257, -0.5651, -0.6205, -0.69

Progress Summary: Semi-automated data loading procedure and additional convolutional-pooling layer to the CNN, reducing the output nodes that are fed to RNN by a factor of 10. Significant improvement in terms of results is observed after only 2 epochs, but it is as computationally expensive as before.
A not-saved run over 8 epochs with this configuration yielded an average RMSE of 1.5 over the training data set. This is still not close enough to the target, and overfitting may occur, but it goes to show that the new architecture is much more promising.